In [1]:
import tensorflow as tf
import shutil

C:\Users\supre.000\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# setup feature columns
domain = tf.feature_column.categorical_column_with_hash_bucket(
 "domain", 100000, dtype=tf.string)
hour = tf.feature_column.categorical_column_with_identity("hour", 24)
device_type = tf.feature_column.categorical_column_with_vocabulary_list(
 "device_type", vocabulary_list=["desktop", "mobile", "tablet"],
 default_value=0)
feature_columns = [domain, hour, device_type]

In [ ]:
# actual model setup
ftrl = tf.train.FtrlOptimizer(
    learning_rate=0.1,
    learning_rate_power=-0.5,
    l1_regularization_strength=0.001,
    l2_regularization_strength=0.0
    )

model_dir = "."
estimator = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    optimizer=ftrl,
    model_dir=model_dir
    )


def input_fn(paths):
    """ model input function """

    names = ["domain", "hour", "device_type", "is_click"]
    record_defaults = [[""], [0], ["desktop"], [0]]

    def _parse_csv(rows_string_tensor):
        columns = tf.decode_csv(rows_string_tensor, record_defaults)
        features = dict(zip(names, columns[:-1]))
        labels = columns[-1]
        return features, labels

    def _input_fn():
        dataset = tf.data.TextLineDataset(paths)
        dataset = dataset.map(_parse_csv)
        dataset = dataset.batch(100)
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()
        return features, labels

    return _input_fn


# Train the model.
paths = ["example.csv"]
estimator.train(input_fn=input_fn(paths), steps=None)


# Export our model
columns = [('hour', tf.int64),
           ('domain', tf.string),
           ('device_type', tf.string)]
feature_placeholders = {
 name: tf.placeholder(dtype, [1], name=name + "_placeholder")
 for name, dtype in columns
}
export_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
    feature_placeholders)
path = estimator.export_savedmodel(model_dir, export_input_fn)

# rename export directory
shutil.move(path, "EXPORT")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '.', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000204312119E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized